# Exercise Sheet 7, Task 3
In this assignment, we will implement a neural network “library”, using Python and Numpy. The tool is inspired by PyTorch’s implementation.

This week, you will use our implementation to train our own word embeddings on a small corpus.

The code below is just the neural network we have built so far.

In [1]:
# As always, we keep the stuff from previous weeks, so that the notebook can be run on its own. In your code, you can (and likely should)
# implement the classes in different files and import them were you need them.

import numpy as np
from typing import List, Tuple

#(you need to install this using `pip3 install tqdm`). This is  special version for jupyter notebooks,
# in standard python code you can simply use 'import tqdm'
from tqdm.notebook import tqdm


class Dropout:
    def __init__(self, p=0.5):
        self.p = p

    def forward(self, x: np.array) -> np.array:
        self.mask = np.random.rand(*x.shape) > self.p
        # Scale the mask to even out missing neurons
        x = x * self.mask / self.p
        return x

    def backward(self, grad: np.array = np.array([[1]])) -> np.array:
        # Scale the mask to even out missing neurons
        return grad * self.mask / self.p

# Once again, we take all of this from previous weeks, only the Neural Network itself will be altered, not the individual layers.
class Sigmoid:
    def __init__(self):
        pass

    def non_rounded_sigmoid(self,x : np.array) -> np.array:
        return 1 / (1 + np.exp(-x))


    def forward(self, x: np.array) -> np.array:
        return 1 / (1 + np.exp(-x))

    def backward(self, x: np.array, grad: np.array = np.array([[1]])) -> np.array:
        return grad * (self.forward(x) * (1 - self.forward(x)))

class MeanSquaredError:
    def __init__(self):
        pass

    def forward(self, y_pred: np.array, y_true: np.array) -> float:
        return np.mean(0.5 * (y_true - y_pred) ** 2)

    def backward(self, y_pred: np.array, y_true: np.array, grad: np.array = np.array([[1]])) -> np.array:
        return  grad * (y_pred - y_true)

class FullyConnectedLayer:
    def __init__(self, input_size: int, output_size: int):
        self.input_size = input_size
        self.output_size = output_size

        self.weights = np.random.randn(self.input_size, self.output_size)
        self.bias = np.zeros((1, self.output_size))

    def forward(self, x: np.array) -> np.array:
        return np.matmul(x, self.weights) + self.bias

    def backward(self, x: np.array, grad: np.array = np.array([[1]])) -> Tuple[np.array,np.array,np.array]:
        x_grad = np.matmul(grad, self.weights.T)
        W_grad = np.matmul(x.T, grad)
        b_grad = grad

        return x_grad, W_grad, b_grad

class NeuralNetwork:
    def __init__(self,
                 input_size: int,
                 output_size: int,
                 hidden_sizes: List[int],
                 activation=Sigmoid,
                 dropout:float =0.5 ):
        self.activ_inputs = None
        self.layer_inputs = None
        s = [input_size] + hidden_sizes + [output_size]
        self.layers = [FullyConnectedLayer(s[i], s[i+1]) for i in range(len(s) - 1)]
        self.dropouts = [Dropout(dropout) for i in range(len(s) - 2)]
        self.activation = activation()

    def forward(self, x: np.array) -> np.array:
        # we need to edit this function to cache our inputs and outputs for each layer during the forward passe!
        self.layer_inputs = []
        self.activ_inputs = []

        for layer,dropout in zip(self.layers[:-1],self.dropouts):
            self.layer_inputs.append(x)
            x = layer.forward(x)
            self.activ_inputs.append(x)
            x = self.activation.forward(x)
            # Dropout Layer
            x = dropout.forward(x)

        #The last layer should not be using an activation function
        self.layer_inputs.append(x)
        x = self.layers[-1].forward(x)
        return x

    def backward(self, x: np.array, grad: np.array = np.array([[1]])) -> Tuple[np.array]:
        W_grads = []
        b_grads = []

        # Backward pass for the last layer
        grad, W_grad, b_grad = self.layers[-1].backward(self.layer_inputs[-1], grad)
        W_grads.append(W_grad)
        b_grads.append(b_grad)

        # Backward pass for the remaining layers
        for i in reversed(range(len(self.activ_inputs))):
            # Dropout Layer
            grad = self.dropouts[i].backward(grad)
            grad = self.activation.backward(self.activ_inputs[i], grad)
            grad, W_grad, b_grad = self.layers[i].backward(self.layer_inputs[i], grad)
            W_grads.append(W_grad)
            b_grads.append(b_grad)

        return grad, list(reversed(W_grads)), list(reversed(b_grads))

In [2]:
class SGD:
    def __init__(self, nn:object, lr:float):
        self.nn = nn
        self.lr = lr

    def update(self,W_grads: List[np.array],b_grads: List[np.array]):
        for i in range(len(self.nn.layers)):
            self.nn.layers[i].weights -= self.lr * W_grads[i]
            self.nn.layers[i].bias -= self.lr * b_grads[i]

class Momentum:
    def __init__(self, nn: object, lr: float, mu: float):
        self.v_W = [0] * len(nn.layers)
        self.v_b = [0] * len(nn.layers)

        self.nn = nn
        self.lr = lr
        self.mu = mu

    def update(self, W_grads: List[np.array], b_grads: List[np.array]):
        for i in range(len(self.nn.layers)):
            self.v_W[i] = self.mu * self.v_W[i] - self.lr * W_grads[i]
            self.v_b[i] = self.mu * self.v_b[i] - self.lr * b_grads[i]

            self.nn.layers[i].weights += self.v_W[i]
            self.nn.layers[i].bias += self.v_b[i]

class Adam:
    def __init__(self,nn: object, lr: float, beta1: float, beta2: float):
        self.m_W = [0] * len(nn.layers)
        self.m_b = [0] * len(nn.layers)

        self.v_W = [0] * len(nn.layers)
        self.v_b = [0] * len(nn.layers)

        self.nn = nn
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2

    def update(self, W_grads: List[np.array], b_grads: List[np.array]):
        for i in range(len(self.nn.layers)):
            self.m_W[i] = self.beta1 * self.m_W[i] + (1 - self.beta1) * W_grads[i]
            self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * b_grads[i]

            self.v_W[i] = self.beta2 * self.v_W[i] + (1 - self.beta2) * W_grads[i] ** 2
            self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * b_grads[i] ** 2

            self.nn.layers[i].weights -= self.lr * self.m_W[i] / (np.sqrt(self.v_W[i]) + 1e-8)
            self.nn.layers[i].bias -= self.lr * self.m_b[i] / (np.sqrt(self.v_b[i]) + 1e-8)

Now, let us build out tuples form the text8 file  from https://data.deepai.org/text8.zip. Download it and unzip it in the same directory as this notebook. This file has already been preprocessed to some degree, so we can just read it in and tokenize it by splitting on spaces.

Since we need the size of our vocabulary, let us construct it while building the tuples. We will use a dictionary to map words to integers, and a list to store the tuples.

As the dataset is rather small we can store everything in our working memory. For larger datasets we may want to split the data up into several files and only load the parts we need for training into memory.

In [3]:
class TextDataset:
    def __init__(self,file_path:str,occurence_threshold:int=5):
        self.file_path = file_path
        self.vocabulary = {}
        self.tuples = []

        self._build_vocab_and_tuples()

        # We filter out all words that occur less than occurrence_threshold times.
        self.vocabulary = {k:v for k,v in self.vocabulary.items() if v >= occurence_threshold}

        # We also need a mapping between the index of a word in the 1-hot vector and the word itself.
        # We also add an <UNK> token for words that are not in our vocabulary, which becomes important if we filter out rare words.
        self.idx_to_word = {0: "<UNK>"}
        self.word_to_idx = {"<UNK>": 0}

        # We construct the mapping based on the alphabetically sorted vocabulary:
        vocab_sorted = sorted(self.vocabulary.keys())
        for token in vocab_sorted:
            self.idx_to_word[len(self.idx_to_word)] = token
            self.word_to_idx[token] = len(self.word_to_idx)
        #Let's add the <UNK> token to the vocabulary, so that the size of the vocabulary is the same as the size of the 1-hot vector.
        self.vocabulary["<UNK>"] = 0


    def _build_vocab_and_tuples(self):
        with open(self.file_path) as f:
            for line in f:
                full_line = line.strip().split()
                for idx,word in enumerate(tqdm(full_line)):
                    if word not in self.vocabulary:
                        self.vocabulary[word] = 1
                    else :
                        self.vocabulary[word] += 1
                    # we want to add the previous 2 words, as well as the following two words to the tuple.
                    # Let us simply discard the samples where this is not possible, i.e. the first and last 2 words.
                    if idx < 2 or idx > len(full_line) - 3:
                        continue
                    else:
                        self.tuples.append(full_line[idx-2:idx+3])


    # To stick close to our inspiration in pytorch, let us implement the __len__ and __getitem__ methods as we would do for a pytorch dataset.
    def __len__(self):
        return len(self.tuples)

    def __getitem__(self, idx):
        return self.tuples[idx]

    def vocabulary_size(self):
        return len(self.vocabulary)

#Here we instantiate our dataset. The second parameter is the minimum number of occurrences a word must have to be included in the vocabulary. This is a simple way to reduce the size of our vocabulary, whitout losing too much information. Depending on the size of your dataset, you may want to increase this number. Also, take a moment to consider the impact this might have on the training process and the resulting embeddings. Are there cases where this might be a bad idea and another approach should be considered?
dataset = TextDataset("text8",0)

  0%|          | 0/17005207 [00:00<?, ?it/s]

We can now investigate our corpus a little bit. Let's check how big our vocabulary is, how many words we have in total, and let's check and confirm that our tuples look right.

In [4]:
import random
print("Vocabulary Size: {}".format(dataset.vocabulary_size()))
print("Total number of words: {}".format(sum(dataset.vocabulary.values())))
print("Total number of tuples: {}".format(len(dataset)))
print("As we would expect, the sum of all word counts is exactly four more than the number of tuples, as we discard the first and last two words in each sentence. Of course this changes as soon as we introduce more preprocessing.\n\nNow let's check the first 10 tuples:\n")
for i in range(10):
    print("Tuple {}: {}".format(i,dataset[i]))
print("\nThey look good. For good measure, let's also quickly check the most common words in our corpus:\n")
for word,count in sorted(dataset.vocabulary.items(), key=lambda x: x[1], reverse=True)[:10]:
    print("{}: {}".format(word,count))

print("\nPretty much what one would expect from a corpus containing actual text samples. For simplicity, we will not strip stopwords from our corpus today, but we may want to consider it in many cases, to remove noise from the corpus.\n\nFinally, let's check the mapping between words and indices, by looking at the first few:\n")
for i in range(10):
    print("Word {}: {}".format(i,dataset.idx_to_word[i]))
print("\n :( These do not look like real english words. Additional preprocessing should be considered to remove repeated characters, or to map words to their stems, but we will not do that today.")

Vocabulary Size: 253855
Total number of words: 17005207
Total number of tuples: 17005203
As we would expect, the sum of all word counts is exactly four more than the number of tuples, as we discard the first and last two words in each sentence. Of course this changes as soon as we introduce more preprocessing.

Now let's check the first 10 tuples:

Tuple 0: ['anarchism', 'originated', 'as', 'a', 'term']
Tuple 1: ['originated', 'as', 'a', 'term', 'of']
Tuple 2: ['as', 'a', 'term', 'of', 'abuse']
Tuple 3: ['a', 'term', 'of', 'abuse', 'first']
Tuple 4: ['term', 'of', 'abuse', 'first', 'used']
Tuple 5: ['of', 'abuse', 'first', 'used', 'against']
Tuple 6: ['abuse', 'first', 'used', 'against', 'early']
Tuple 7: ['first', 'used', 'against', 'early', 'working']
Tuple 8: ['used', 'against', 'early', 'working', 'class']
Tuple 9: ['against', 'early', 'working', 'class', 'radicals']

They look good. For good measure, let's also quickly check the most common words in our corpus:

the: 1061396
of: 5

In [5]:
print("\n\nLet's also check some random words from the vocabulary, their indices and counts:\n")
for i in range(10):
    word = random.choice(list(dataset.vocabulary.keys()))
    print("Word: {}, Index: {}, Count: {}".format(word,dataset.word_to_idx[word],dataset.vocabulary[word]))
print("\n\nNow that we have our dataset, we can start and train our model.\n")



Let's also check some random words from the vocabulary, their indices and counts:

Word: andfrance, Index: 9218, Count: 1
Word: sensitive, Index: 203200, Count: 425
Word: arjuna, Index: 13362, Count: 32
Word: hird, Index: 101174, Count: 10
Word: renouced, Index: 190533, Count: 1
Word: electrostatic, Index: 68825, Count: 65
Word: mahaparinirvana, Index: 136744, Count: 4
Word: acetaminophen, Index: 1580, Count: 12
Word: kiyonaga, Index: 122042, Count: 1
Word: halococcus, Index: 95830, Count: 1


Now that we have our dataset, we can start and train our model.



In [ ]:
vocab_size = dataset.vocabulary_size()
sample_count = len(dataset)
index_list = list(range(sample_count))

# Network Initialization
net = NeuralNetwork(vocab_size, vocab_size,[32], Sigmoid)

# Loss
loss_function = MeanSquaredError()

# Optimizer
optimizer = SGD(net, lr=0.1)
# optimizer = Momentum(net, lr=0.1, mu=0.0)
# optimizer = Adam(net, 0.001, 0.9, 0.999)

epochs = 3

# tqdm (you need to install it using `pip3 install tqdm`) will show a progress bar for the loop
for epoch_idx, epoch in enumerate(tqdm(range(epochs))):
    # It is always a good idea to shuffle the dataset.
    # Here we shuffle our indices and then iterate over them
    np.random.shuffle(index_list)

    #training over all samples may take waaaay to long, especially on a CPU, so we will only train on 10k random samples per batch
    for i in tqdm(index_list[:10000]):
        # get our sample at the index location
        sample = dataset[i]
        # get our center word and context from that sample and let us construct our 1-hot vector
        # first, we build a vector of the correct shape, initialized with zeros
        x = np.zeros((1,vocab_size))
        # then we set the index of the center word to 1
        x[0,dataset.word_to_idx.get(sample[2],0)] = 1
        # now we do the same for the context words, we use the get function of dictionaries to return 0 if the word is not in the dictionary, as that is the index of the <UNK> token.
        y = np.zeros((1,vocab_size))
        y[0,dataset.word_to_idx.get(sample[0],0)] = 1
        y[0,dataset.word_to_idx.get(sample[1],0)] = 1
        y[0,dataset.word_to_idx.get(sample[3],0)] = 1
        y[0,dataset.word_to_idx.get(sample[4],0)] = 1

        # Forward Pass
        pred = net.forward(x)

        # Loss Calculation
        loss = loss_function.forward(pred, y)

        # Backward Pass
        grad = loss_function.backward(pred, y)
        grad, W_grads, b_grads = net.backward(x, grad)

        # Update
        optimizer.update(W_grads, b_grads)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

/tmp/ipykernel_78639/4208183115.py:36: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


This takes forever. Preprocessing can greatly reduce the size of the dataset, since it dictates the size of our input and output layers. One easy way to achieve this is by dropping all rare words from the dataset. I have prepared our dataset to do. Try building the dataset with only words that occur more than 5/50/500 times. Take a look at how the vocabulary changes, and the impact it has on our training. Also, feel free to implement you own preprocessing, dropping rare words is a very simple approach and, while effective, not the only nor always the ideal choice.

Once we are dne training our model we can take a look at the embeddings. We can do this by simply looking at the weights of the hidden layer. We can also use the embeddings to find similar words. We can do this by calculating the cosine similarity between the embeddings of two words. Let's try it out.